Synthetic Data Set Generator


In [ ]:
!pip3 install openai gradio python-dotenv jupyter-black

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import gradio as gr
from typing import List, Dict
import random


# Load environment variables
load_dotenv()

# Initialize OpenAI client pointing to OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

ollama_client = OpenAI(
    base_url="http://localhost:11434/v1",  # Ollama's local endpoint
    api_key="ollama",  # Ollama doesn't need a real API key, but the SDK requires something
)

In [ ]:
# Test the connection with a simple call
response = client.chat.completions.create(
    model="anthropic/claude-3.5-sonnet",  # You can change this to other models
    messages=[
        {
            "role": "user",
            "content": "Say 'Hello! Connection successful!' if you can read this.",
        }
    ],
)

print(response.choices[0].message.content)

In [ ]:
def generate_support_conversation(
    domain: str,
    issue_type: str,
    conversation_length: str,
    tone: str,
    model: str = "anthropic/claude-3.5-sonnet",
    use_ollama: bool = False,
) -> Dict:
    """
    Generate a synthetic customer support conversation.

    Args:
        domain: Business domain (e-commerce, SaaS, banking, etc.)
        issue_type: Type of customer issue
        conversation_length: short, medium, or long
        tone: friendly, professional, frustrated, etc.
        model: The model to use
        use_ollama: Whether to use local Ollama instead of OpenRouter

    Returns:
        Dictionary containing the conversation and metadata
    """

    # Map conversation length to number of exchanges
    length_map = {"short": "3-4", "medium": "5-7", "long": "8-12"}

    num_exchanges = length_map.get(conversation_length, "5-7")

    # Create the prompt
    prompt = f"""Generate a realistic customer support conversation for a {domain} company.

Issue Type: {issue_type}
Customer Tone: {tone}
Number of message exchanges: {num_exchanges} (customer and agent messages combined)

Requirements:
- Make it realistic with natural language, including minor typos or informal language from the customer
- The agent should be helpful and professional
- Include a resolution or clear next steps
- Format as JSON with this structure:
{{
    "conversation": [
        {{"speaker": "customer", "message": "..."}},
        {{"speaker": "agent", "message": "..."}}
    ],
    "metadata": {{
        "issue_category": "...",
        "resolution_status": "resolved/pending/escalated",
        "customer_satisfaction": "positive/neutral/negative"
    }}
}}

Generate only the JSON, no other text."""

    try:
        # Choose which client to use
        active_client = ollama_client if use_ollama else client

        response = active_client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,
        )

        # Parse the response
        content = response.choices[0].message.content

        # Clean up potential markdown formatting
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()

        conversation_data = json.loads(content)
        return conversation_data

    except Exception as e:
        return {"error": str(e), "conversation": [], "metadata": {}}

In [ ]:
def format_conversation_for_display(conversation_data: Dict) -> tuple:
    """
    Format the conversation data for Gradio display.

    Returns:
        Tuple of (chatbot_messages, metadata_text, json_output)
    """
    if "error" in conversation_data:
        return (
            [],
            f"Error: {conversation_data['error']}",
            json.dumps(conversation_data, indent=2),
        )

    # Format for Gradio Chatbot component (list of [user_msg, bot_msg] pairs)
    chatbot_messages = []
    conversation = conversation_data.get("conversation", [])

    # Pair up customer and agent messages
    i = 0
    while i < len(conversation):
        customer_msg = None
        agent_msg = None

        if i < len(conversation) and conversation[i]["speaker"] == "customer":
            customer_msg = conversation[i]["message"]
            i += 1

        if i < len(conversation) and conversation[i]["speaker"] == "agent":
            agent_msg = conversation[i]["message"]
            i += 1

        chatbot_messages.append([customer_msg, agent_msg])

    # Format metadata
    metadata = conversation_data.get("metadata", {})
    metadata_text = f"""**Metadata:**
- Issue Category: {metadata.get('issue_category', 'N/A')}
- Resolution Status: {metadata.get('resolution_status', 'N/A')}
- Customer Satisfaction: {metadata.get('customer_satisfaction', 'N/A')}
"""

    # JSON output
    json_output = json.dumps(conversation_data, indent=2)

    return chatbot_messages, metadata_text, json_output


def generate_conversation_ui(domain, issue_type, length, tone, model):
    """Wrapper function for Gradio interface"""
    conversation_data = generate_support_conversation(
        domain=domain,
        issue_type=issue_type,
        conversation_length=length,
        tone=tone,
        model=model,
    )

    return format_conversation_for_display(conversation_data)


# Create the Gradio interface
with gr.Blocks(title="Customer Support Conversation Generator") as demo:
    gr.Markdown("# 🎭 Synthetic Customer Support Conversation Generator")
    gr.Markdown(
        "Generate realistic customer support conversations for training, testing, or demonstrations."
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Configuration")

            domain = gr.Dropdown(
                choices=[
                    "E-commerce",
                    "SaaS",
                    "Banking",
                    "Telecommunications",
                    "Healthcare",
                    "Travel",
                ],
                value="E-commerce",
                label="Business Domain",
            )

            issue_type = gr.Dropdown(
                choices=[
                    "Refund Request",
                    "Technical Problem",
                    "Account Access",
                    "Billing Question",
                    "Product Inquiry",
                    "Shipping Issue",
                    "Feature Request",
                    "Complaint",
                ],
                value="Refund Request",
                label="Issue Type",
            )

            length = gr.Radio(
                choices=["short", "medium", "long"],
                value="medium",
                label="Conversation Length",
            )

            tone = gr.Dropdown(
                choices=[
                    "Friendly",
                    "Professional",
                    "Frustrated",
                    "Confused",
                    "Urgent",
                ],
                value="Friendly",
                label="Customer Tone",
            )

            model = gr.Dropdown(
                choices=[
                    "anthropic/claude-3.5-sonnet",
                    "openai/gpt-4o",
                    "google/gemini-pro",
                    "meta-llama/llama-3.1-70b-instruct",
                ],
                value="anthropic/claude-3.5-sonnet",
                label="Model",
            )

            generate_btn = gr.Button("Generate Conversation", variant="primary")

        with gr.Column(scale=2):
            gr.Markdown("### Generated Conversation")
            chatbot = gr.Chatbot(label="Conversation", height=400)
            metadata_display = gr.Markdown("*Generate a conversation to see metadata*")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### JSON Output")
            json_output = gr.Code(label="Raw JSON", language="json", lines=15)

    # Connect the generate button
    generate_btn.click(
        fn=generate_conversation_ui,
        inputs=[domain, issue_type, length, tone, model],
        outputs=[chatbot, metadata_display, json_output],
    )

    gr.Markdown(
        """
    ### Tips:
    - Try different models to compare output quality and style
    - Experiment with different tones to see how it affects the conversation
    - Use the JSON output to integrate into your applications
    """
    )

# Launch the interface
demo.launch()

In [ ]:
import pandas as pd
from datetime import datetime


def generate_batch_conversations(
    domain: str,
    issue_type: str,
    length: str,
    tone: str,
    model: str,
    num_conversations: int,
) -> tuple:
    """Generate multiple conversations at once"""

    all_conversations = []
    progress_updates = []

    for i in range(num_conversations):
        progress_updates.append(f"Generating conversation {i+1}/{num_conversations}...")

        conversation_data = generate_support_conversation(
            domain=domain,
            issue_type=issue_type,
            conversation_length=length,
            tone=tone,
            model=model,
        )

        # Add an ID and timestamp
        conversation_data["id"] = (
            f"conv_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{i+1}"
        )
        conversation_data["generated_at"] = datetime.now().isoformat()
        conversation_data["config"] = {
            "domain": domain,
            "issue_type": issue_type,
            "length": length,
            "tone": tone,
            "model": model,
        }

        all_conversations.append(conversation_data)

    # Create a summary
    summary = f"""### Batch Generation Complete!
    
**Generated:** {num_conversations} conversations
**Domain:** {domain}
**Issue Type:** {issue_type}
**Model:** {model}

You can now download the results as JSON or CSV.
"""

    # Convert to JSON string for download
    json_str = json.dumps(all_conversations, indent=2)

    # Create CSV format (flattened data)
    csv_data = []
    for conv in all_conversations:
        # Flatten conversation into a single row
        messages = conv.get("conversation", [])
        full_conversation = "\n".join(
            [f"{msg['speaker']}: {msg['message']}" for msg in messages]
        )

        csv_data.append(
            {
                "id": conv.get("id"),
                "domain": domain,
                "issue_type": issue_type,
                "tone": tone,
                "conversation": full_conversation,
                "issue_category": conv.get("metadata", {}).get("issue_category"),
                "resolution_status": conv.get("metadata", {}).get("resolution_status"),
                "customer_satisfaction": conv.get("metadata", {}).get(
                    "customer_satisfaction"
                ),
                "generated_at": conv.get("generated_at"),
            }
        )

    df = pd.DataFrame(csv_data)
    csv_str = df.to_csv(index=False)

    return summary, json_str, csv_str


# Enhanced Gradio interface with batch generation
with gr.Blocks(
    title="Customer Support Conversation Generator", theme=gr.themes.Soft()
) as demo:
    gr.Markdown("# 🎭 Synthetic Customer Support Conversation Generator")
    gr.Markdown(
        "Generate realistic customer support conversations for training, testing, or demonstrations."
    )

    with gr.Tabs():
        # Tab 1: Single Generation
        with gr.Tab("Single Conversation"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Configuration")

                    domain_single = gr.Dropdown(
                        choices=[
                            "E-commerce",
                            "SaaS",
                            "Banking",
                            "Telecommunications",
                            "Healthcare",
                            "Travel",
                        ],
                        value="E-commerce",
                        label="Business Domain",
                    )

                    issue_type_single = gr.Dropdown(
                        choices=[
                            "Refund Request",
                            "Technical Problem",
                            "Account Access",
                            "Billing Question",
                            "Product Inquiry",
                            "Shipping Issue",
                            "Feature Request",
                            "Complaint",
                        ],
                        value="Refund Request",
                        label="Issue Type",
                    )

                    length_single = gr.Radio(
                        choices=["short", "medium", "long"],
                        value="medium",
                        label="Conversation Length",
                    )

                    tone_single = gr.Dropdown(
                        choices=[
                            "Friendly",
                            "Professional",
                            "Frustrated",
                            "Confused",
                            "Urgent",
                        ],
                        value="Friendly",
                        label="Customer Tone",
                    )

                    model_single = gr.Dropdown(
                        choices=[
                            "anthropic/claude-3.5-sonnet",
                            "openai/gpt-4o",
                            "google/gemini-pro",
                            "meta-llama/llama-3.1-70b-instruct",
                        ],
                        value="anthropic/claude-3.5-sonnet",
                        label="Model",
                    )

                    generate_btn_single = gr.Button(
                        "Generate Conversation", variant="primary"
                    )

                with gr.Column(scale=2):
                    gr.Markdown("### Generated Conversation")
                    chatbot_single = gr.Chatbot(label="Conversation", height=400)
                    metadata_display_single = gr.Markdown(
                        "*Generate a conversation to see metadata*"
                    )

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### JSON Output")
                    json_output_single = gr.Code(
                        label="Raw JSON", language="json", lines=15
                    )

            generate_btn_single.click(
                fn=generate_conversation_ui,
                inputs=[
                    domain_single,
                    issue_type_single,
                    length_single,
                    tone_single,
                    model_single,
                ],
                outputs=[chatbot_single, metadata_display_single, json_output_single],
            )

        # Tab 2: Batch Generation
        with gr.Tab("Batch Generation"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Batch Configuration")

                    num_conversations = gr.Slider(
                        minimum=2,
                        maximum=20,
                        value=5,
                        step=1,
                        label="Number of Conversations",
                    )

                    domain_batch = gr.Dropdown(
                        choices=[
                            "E-commerce",
                            "SaaS",
                            "Banking",
                            "Telecommunications",
                            "Healthcare",
                            "Travel",
                        ],
                        value="E-commerce",
                        label="Business Domain",
                    )

                    issue_type_batch = gr.Dropdown(
                        choices=[
                            "Refund Request",
                            "Technical Problem",
                            "Account Access",
                            "Billing Question",
                            "Product Inquiry",
                            "Shipping Issue",
                            "Feature Request",
                            "Complaint",
                        ],
                        value="Refund Request",
                        label="Issue Type",
                    )

                    length_batch = gr.Radio(
                        choices=["short", "medium", "long"],
                        value="medium",
                        label="Conversation Length",
                    )

                    tone_batch = gr.Dropdown(
                        choices=[
                            "Friendly",
                            "Professional",
                            "Frustrated",
                            "Confused",
                            "Urgent",
                        ],
                        value="Friendly",
                        label="Customer Tone",
                    )

                    model_batch = gr.Dropdown(
                        choices=[
                            "anthropic/claude-3.5-sonnet",
                            "openai/gpt-4o",
                            "google/gemini-pro",
                            "meta-llama/llama-3.1-70b-instruct",
                        ],
                        value="anthropic/claude-3.5-sonnet",
                        label="Model",
                    )

                    generate_btn_batch = gr.Button(
                        "Generate Batch", variant="primary", size="lg"
                    )

                with gr.Column(scale=2):
                    summary_output = gr.Markdown(
                        "*Configure and generate a batch of conversations*"
                    )

                    with gr.Row():
                        json_download = gr.File(label="Download JSON")
                        csv_download = gr.File(label="Download CSV")

            def generate_and_save_batch(domain, issue_type, length, tone, model, num):
                summary, json_str, csv_str = generate_batch_conversations(
                    domain, issue_type, length, tone, model, num
                )

                # Save to temporary files
                json_filename = (
                    f"conversations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                )
                csv_filename = (
                    f"conversations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
                )

                with open(json_filename, "w") as f:
                    f.write(json_str)

                with open(csv_filename, "w") as f:
                    f.write(csv_str)

                return summary, json_filename, csv_filename

            generate_btn_batch.click(
                fn=generate_and_save_batch,
                inputs=[
                    domain_batch,
                    issue_type_batch,
                    length_batch,
                    tone_batch,
                    model_batch,
                    num_conversations,
                ],
                outputs=[summary_output, json_download, csv_download],
            )

    gr.Markdown(
        """
    ### Tips:
    - **Single Conversation:** Perfect for testing and seeing immediate results
    - **Batch Generation:** Generate multiple conversations at once for training datasets
    - Try different models to compare output quality and style
    - Export to JSON for application integration or CSV for analysis
    """
    )

# Launch the enhanced interface
demo.launch()

In [ ]:
def generate_conversation_ui(domain, issue_type, length, tone, model, provider):
    """Wrapper function for Gradio interface"""
    use_ollama = provider == "Ollama (Local)"

    conversation_data = generate_support_conversation(
        domain=domain,
        issue_type=issue_type,
        conversation_length=length,
        tone=tone,
        model=model,
        use_ollama=use_ollama,
    )

    return format_conversation_for_display(conversation_data)


def generate_batch_conversations(
    domain: str,
    issue_type: str,
    length: str,
    tone: str,
    model: str,
    num_conversations: int,
    provider: str,
) -> tuple:
    """Generate multiple conversations at once"""

    use_ollama = provider == "Ollama (Local)"
    all_conversations = []

    for i in range(num_conversations):
        conversation_data = generate_support_conversation(
            domain=domain,
            issue_type=issue_type,
            conversation_length=length,
            tone=tone,
            model=model,
            use_ollama=use_ollama,
        )

        # Add an ID and timestamp
        conversation_data["id"] = (
            f"conv_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{i+1}"
        )
        conversation_data["generated_at"] = datetime.now().isoformat()
        conversation_data["config"] = {
            "domain": domain,
            "issue_type": issue_type,
            "length": length,
            "tone": tone,
            "model": model,
            "provider": provider,
        }

        all_conversations.append(conversation_data)

    # Create a summary
    summary = f"""### Batch Generation Complete!
    
**Generated:** {num_conversations} conversations
**Provider:** {provider}
**Domain:** {domain}
**Issue Type:** {issue_type}
**Model:** {model}

You can now download the results as JSON or CSV.
"""

    # Convert to JSON string for download
    json_str = json.dumps(all_conversations, indent=2)

    # Create CSV format (flattened data)
    csv_data = []
    for conv in all_conversations:
        messages = conv.get("conversation", [])
        full_conversation = "\n".join(
            [f"{msg['speaker']}: {msg['message']}" for msg in messages]
        )

        csv_data.append(
            {
                "id": conv.get("id"),
                "domain": domain,
                "issue_type": issue_type,
                "tone": tone,
                "conversation": full_conversation,
                "issue_category": conv.get("metadata", {}).get("issue_category"),
                "resolution_status": conv.get("metadata", {}).get("resolution_status"),
                "customer_satisfaction": conv.get("metadata", {}).get(
                    "customer_satisfaction"
                ),
                "generated_at": conv.get("generated_at"),
            }
        )

    df = pd.DataFrame(csv_data)
    csv_str = df.to_csv(index=False)

    return summary, json_str, csv_str


# Enhanced Gradio interface with Ollama support
with gr.Blocks(
    title="Customer Support Conversation Generator", theme=gr.themes.Soft()
) as demo:
    gr.Markdown("# 🎭 Synthetic Customer Support Conversation Generator")
    gr.Markdown(
        "Generate realistic customer support conversations using OpenRouter or local Ollama models."
    )

    with gr.Tabs():
        # Tab 1: Single Generation
        with gr.Tab("Single Conversation"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Configuration")

                    provider_single = gr.Radio(
                        choices=["OpenRouter (Cloud)", "Ollama (Local)"],
                        value="OpenRouter (Cloud)",
                        label="Provider",
                    )

                    model_single = gr.Dropdown(
                        choices=[
                            "anthropic/claude-3.5-sonnet",
                            "openai/gpt-4o",
                            "google/gemini-pro",
                            "meta-llama/llama-3.1-70b-instruct",
                        ],
                        value="anthropic/claude-3.5-sonnet",
                        label="Model (OpenRouter)",
                    )

                    ollama_model_single = gr.Textbox(
                        value="llama3.1",
                        label="Model (Ollama)",
                        placeholder="e.g., llama3.1, mistral, qwen2.5",
                    )

                    domain_single = gr.Dropdown(
                        choices=[
                            "E-commerce",
                            "SaaS",
                            "Banking",
                            "Telecommunications",
                            "Healthcare",
                            "Travel",
                        ],
                        value="E-commerce",
                        label="Business Domain",
                    )

                    issue_type_single = gr.Dropdown(
                        choices=[
                            "Refund Request",
                            "Technical Problem",
                            "Account Access",
                            "Billing Question",
                            "Product Inquiry",
                            "Shipping Issue",
                            "Feature Request",
                            "Complaint",
                        ],
                        value="Refund Request",
                        label="Issue Type",
                    )

                    length_single = gr.Radio(
                        choices=["short", "medium", "long"],
                        value="medium",
                        label="Conversation Length",
                    )

                    tone_single = gr.Dropdown(
                        choices=[
                            "Friendly",
                            "Professional",
                            "Frustrated",
                            "Confused",
                            "Urgent",
                        ],
                        value="Friendly",
                        label="Customer Tone",
                    )

                    generate_btn_single = gr.Button(
                        "Generate Conversation", variant="primary"
                    )

                with gr.Column(scale=2):
                    gr.Markdown("### Generated Conversation")
                    chatbot_single = gr.Chatbot(label="Conversation", height=400)
                    metadata_display_single = gr.Markdown(
                        "*Generate a conversation to see metadata*"
                    )

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### JSON Output")
                    json_output_single = gr.Code(
                        label="Raw JSON", language="json", lines=15
                    )

            def generate_with_provider_single(
                domain, issue_type, length, tone, model, ollama_model, provider
            ):
                selected_model = ollama_model if provider == "Ollama (Local)" else model
                return generate_conversation_ui(
                    domain, issue_type, length, tone, selected_model, provider
                )

            generate_btn_single.click(
                fn=generate_with_provider_single,
                inputs=[
                    domain_single,
                    issue_type_single,
                    length_single,
                    tone_single,
                    model_single,
                    ollama_model_single,
                    provider_single,
                ],
                outputs=[chatbot_single, metadata_display_single, json_output_single],
            )

        # Tab 2: Batch Generation
        with gr.Tab("Batch Generation"):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### Batch Configuration")

                    num_conversations = gr.Slider(
                        minimum=2,
                        maximum=20,
                        value=5,
                        step=1,
                        label="Number of Conversations",
                    )

                    provider_batch = gr.Radio(
                        choices=["OpenRouter (Cloud)", "Ollama (Local)"],
                        value="OpenRouter (Cloud)",
                        label="Provider",
                    )

                    model_batch = gr.Dropdown(
                        choices=[
                            "anthropic/claude-3.5-sonnet",
                            "openai/gpt-4o",
                            "google/gemini-pro",
                            "meta-llama/llama-3.1-70b-instruct",
                        ],
                        value="anthropic/claude-3.5-sonnet",
                        label="Model (OpenRouter)",
                    )

                    ollama_model_batch = gr.Textbox(
                        value="llama3.1",
                        label="Model (Ollama)",
                        placeholder="e.g., llama3.1, mistral, qwen2.5",
                    )

                    domain_batch = gr.Dropdown(
                        choices=[
                            "E-commerce",
                            "SaaS",
                            "Banking",
                            "Telecommunications",
                            "Healthcare",
                            "Travel",
                        ],
                        value="E-commerce",
                        label="Business Domain",
                    )

                    issue_type_batch = gr.Dropdown(
                        choices=[
                            "Refund Request",
                            "Technical Problem",
                            "Account Access",
                            "Billing Question",
                            "Product Inquiry",
                            "Shipping Issue",
                            "Feature Request",
                            "Complaint",
                        ],
                        value="Refund Request",
                        label="Issue Type",
                    )

                    length_batch = gr.Radio(
                        choices=["short", "medium", "long"],
                        value="medium",
                        label="Conversation Length",
                    )

                    tone_batch = gr.Dropdown(
                        choices=[
                            "Friendly",
                            "Professional",
                            "Frustrated",
                            "Confused",
                            "Urgent",
                        ],
                        value="Friendly",
                        label="Customer Tone",
                    )

                    generate_btn_batch = gr.Button(
                        "Generate Batch", variant="primary", size="lg"
                    )

                with gr.Column(scale=2):
                    summary_output = gr.Markdown(
                        "*Configure and generate a batch of conversations*"
                    )

                    with gr.Row():
                        json_download = gr.File(label="Download JSON")
                        csv_download = gr.File(label="Download CSV")

            def generate_and_save_batch_with_provider(
                domain, issue_type, length, tone, model, ollama_model, provider, num
            ):
                selected_model = ollama_model if provider == "Ollama (Local)" else model
                summary, json_str, csv_str = generate_batch_conversations(
                    domain, issue_type, length, tone, selected_model, num, provider
                )

                # Save to temporary files
                json_filename = (
                    f"conversations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                )
                csv_filename = (
                    f"conversations_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
                )

                with open(json_filename, "w") as f:
                    f.write(json_str)

                with open(csv_filename, "w") as f:
                    f.write(csv_str)

                return summary, json_filename, csv_filename

            generate_btn_batch.click(
                fn=generate_and_save_batch_with_provider,
                inputs=[
                    domain_batch,
                    issue_type_batch,
                    length_batch,
                    tone_batch,
                    model_batch,
                    ollama_model_batch,
                    provider_batch,
                    num_conversations,
                ],
                outputs=[summary_output, json_download, csv_download],
            )

    gr.Markdown(
        """
    ### Tips:
    - **OpenRouter:** Use cloud models (requires API key)
    - **Ollama:** Use local models (requires Ollama running on localhost:11434)
    - Popular Ollama models: `llama3.1`, `mistral`, `qwen2.5`, `phi3`
    - Run `ollama list` in terminal to see your available models
    """
    )

# Launch the enhanced interface
demo.launch()